# IMPORTATION 

In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import Sequential
from gensim.models import KeyedVectors
from keras import optimizers
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


WORD EMBEEDING 

In [ ]:
model_arabic = KeyedVectors.load_word2vec_format('CG_EMBEEDING_CBOW_stable.txt')

Fichier Stop words

In [ ]:
stopf = open('list.txt', 'rt', encoding='utf-8')
alld = stopf.read()
stop_words = alld.split("\n")

Lire les données et ajouter `<start>` et `<end>  `  à chaque question et et réponse 

In [ ]:

#Reponse
file = open('CG_BIG_A.txt', 'rt', encoding='utf-8')
text = file.read()
A_data = text.split("\n")
A_data[0]=A_data[0].replace('\ufeff','')

    

#Question
file = open('CG_BIG_Q.txt', 'rt', encoding='utf-8')
text = file.read()
Q_data = text.split("\n")
Q_data[0]=Q_data[0].replace('\ufeff','')


x = len(A_data)
a = 0
while a < x:
    if(len(A_data[a])>100):
        del A_data[a]
        del Q_data[a]
        x = x-1
    else:
        A_data[a] = '<start> '+A_data[a]+' <end>'
        a = a +1
x = len(Q_data)
a=0
while a < x:
    if(len(Q_data[a])>100):
        del A_data[a]
        del Q_data[a]
        x=x-1
    else:
        Q_data[a] = '<start> '+Q_data[a]+' <end>'
        a = a +1

print('la taille des Questions : {}'.format(len(Q_data)))
print('la taille des Réponses  : {}'.format(len(A_data)))

la taille des Questions : 6498
la taille des Réponses  : 6498


Supprimer les Stop words et ajouter les meme données sans stop words à la base de données 

In [ ]:
from keras_preprocessing.text import text_to_word_sequence
qq = []
for i in range(len(A_data)):
    s = text_to_word_sequence(Q_data[i],filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
    b = ''
    for token in s:
        if token not in stop_words:
            b = b +token+' '
    qq.append(b)

In [ ]:
A_data = A_data + A_data
Q_data = Q_data + qq

#Prétraitement

**Etape 1: TOKENIZE**


*   Son but c’est de symboliser les données, c'est à dire convertie le texte en valeurs numériques **; transformer chaque chaque mot en nombre ;** 
 

*   Cela permet au réseau neuronal d'effectuer des opérations sur les données d'entrée

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer(char_level=False,filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(x)
    
    return tokenizer.texts_to_sequences(x), tokenizer

**Etape 2: PAD**


*   Lorsque nous introduisons nos séquences d'ID de mot dans le modèle finale, les séquence doient avoir la même longueur.
 

*   Pour ce faire, un remplissage est ajouté à toute séquence plus courte que la longueur maximale (c'est-à-dire plus courte que la phrase la plus longue).

In [ ]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
        print('Length Max: {}'.format(length))
        print()
    return pad_sequences(x, maxlen=length, padding='post')

**Etape 3: Prétraitement**


*   La création d'un dictionnaire des `{mot:id}` pour les Questions et les réponses .

*   Transfomer toutes les Questions et les Réponses à des vecteur.

In [ ]:
def preprocess(Q,A):
 
    preprocess_Q, Q_tk = tokenize(Q)
    preprocess_A, A_tk = tokenize(A)

    preprocess_Q = pad(preprocess_Q)
    preprocess_A = pad(preprocess_A)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_A = preprocess_A.reshape(*preprocess_A.shape, 1)

    return preprocess_Q, preprocess_A, Q_tk, A_tk

*   Calcule de la taille du vocabulaire Q et A.

*   Division de la base de données

*   la taille maximale des Q et A

In [ ]:
preproc_Q, preproc_A, Q_tokenizer, A_tokenizer = preprocess(Q_data, A_data)

# spliting our data set
X_train, X_test, Y_train, Y_test = train_test_split(preproc_Q,preproc_A,test_size=0.1)

print(Q_tokenizer.word_index) #Dict Q
print(A_tokenizer.word_index) #Dict A


max_Q_sequence_length = preproc_Q.shape[1]
max_A_sequence_length = preproc_A.shape[1]

Q_vocab_size = len(Q_tokenizer.word_index)+1
A_vocab_size = len(A_tokenizer.word_index)+1

print('\n')

print('Data Preprocessed')
print("Max Q sentence length:", max_Q_sequence_length)
print("Max A sentence length:", max_A_sequence_length)
print("Q vocabulary size:", Q_vocab_size)
print("A vocabulary size:", A_vocab_size)

Length Max: 23

Length Max: 19

{'<start>': 1, '<end>': 2, 'من': 3, 'ما': 4, 'هو': 5, 'اسم': 6, 'في': 7, 'الذي': 8, 'هي': 9, 'أول': 10, 'ماذا': 11, 'التي': 12, 'على': 13, 'العالم': 14, 'الاسم': 15, 'الرسول': 16, 'مدينة': 17, 'كلمة': 18, 'يطلق': 19, 'لقب': 20, 'بن': 21, 'يسمى': 22, 'صاحب': 23, 'أين': 24, 'تسمى': 25, 'الصحابي': 26, 'أكبر': 27, 'تعني': 28, 'ماهي': 29, 'كان': 30, 'معنى': 31, 'أي': 32, 'المدينة': 33, 'بـ': 34, 'الدولة': 35, 'الشاعر': 36, 'ماهو': 37, 'يوجد': 38, 'العرب': 39, 'الله': 40, 'دولة': 41, 'أطلق': 42, 'آخر': 43, 'فما': 44, 'كانت': 45, 'معركة': 46, 'لماذا': 47, 'مصر': 48, 'عاصمة': 49, 'يقع': 50, 'عليه': 51, 'تقع': 52, 'أقدم': 53, 'عند': 54, 'قتل': 55, 'كتاب': 56, 'توجد': 57, 'عهد': 58, 'الملك': 59, 'نهر': 60, 'اللغة': 61, 'مؤسس': 62, 'النبي': 63, 'السلام': 64, 'الحيوان': 65, 'بين': 66, 'فمن': 67, 'الاسلام': 68, 'أبو': 69, 'بعد': 70, 'المادة': 71, 'به': 72, 'القائد': 73, 'ولد': 74, 'الماء': 75, 'الآخر': 76, 'إلى': 77, 'الحرب': 78, 'ملك': 79, 'قائد': 80, 'المسلمين': 81

**Etape 4: Embbedings**


*   l’utilisation d'une matrice qui contient les représentations des mots en vecteurs réalisés avec plusieurs méthodes **(W2V )** ,donc ça sera nécessaire de remplacer chaque mot par un vecteur comme input du model.


*   Et Dans notre cas la taille de chaque vecteur est 256

In [ ]:
def embedding_matrix_creater(embedding_dimention):
  embedding_matrix = np.zeros((len(Q_tokenizer.word_index)+1, embedding_dimention))
  for word, i in Q_tokenizer.word_index.items():
    embedding_vector = None
    embedding_matrix[i] = model_arabic[word]
  return embedding_matrix


mat=embedding_matrix_creater(256)

#Modèle



*   Utilisation de LSTM 
*   Les résultats sont des probabilités à l'aide de la fonction d'activation softmax



In [ ]:
models2s=Sequential()
models2s.add(Embedding(input_dim=Q_vocab_size,
                            output_dim=256,
                            input_length=max_Q_sequence_length,
                            trainable=True,
                            weights=[mat],
                            name='enc_embedding'))

models2s.add(LSTM(256,input_shape=(max_Q_sequence_length,256)))
models2s.add(RepeatVector(max_A_sequence_length))
models2s.add(LSTM(256,return_sequences=True))
models2s.add(TimeDistributed(Dense(units=A_vocab_size+1)))
models2s.add(Activation('softmax', name='softmax'))
models2s.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_embedding (Embedding)    (None, 23, 256)           2711808   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 19, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 19, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 19, 6820)          1752740   
_________________________________________________________________
softmax (Activation)         (None, 19, 6820)          0         
Total params: 5,515,172
Trainable params: 5,515,172
Non-trainable params: 0
____________________________________________

l'entrainement necessite deux fonction. model.compile qui aide le modèle à corriger durant l'entrainement plusieurs paramétres, et model.fit qui lance l'entrainement avec les données d'entrées et d'autres paramétres .

In [ ]:
rms = optimizers.RMSprop(lr=0.002)

models2s.compile(optimizer=rms, loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = models2s.fit( X_train,Y_train, epochs=50, batch_size=64, validation_data=(X_test,Y_test))

c:\users\ucefb\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Train on 11696 samples, validate on 1300 samples
Epoch 1/50
11696/11696 [==============================] - 107s 9ms/step - loss: 1.4005 - accuracy: 0.8374 - val_loss: 1.1994 - val_accuracy: 0.8542
Epoch 2/50
11696/11696 [==============================] - 108s 9ms/step - loss: 1.1281 - accuracy: 0.8559 - val_loss: 1.1553 - val_accuracy: 0.8561
Epoch 3/50
11696/11696 [==============================] - 109s 9ms/step - loss: 1.0677 - accuracy: 0.8613 - val_loss: 1.1316 - val_accuracy: 0.8583
Epoch 4/50
11696/11696 [==============================] - 111s 9ms/step - loss: 0.9986 - accuracy: 0.8689 - val_loss: 1.0882 - val_accuracy: 0.8651
Epoch 5/50
11696/11696 [==============================] - 131s 11ms/step - loss: 0.9393 - accuracy: 0.8743 - val_loss: 1.0414 - val_accuracy: 0.8668
Epoch 6/50
11696/11696 [==============================] - 116s 10ms/step - loss: 0.8948 - accuracy: 0.8767 - val_loss: 1.0108 - val_accuracy: 0.8678
Epoch 7/50
11696/11696 [==============================] - 113



*   toPredectible : transorme la question en entrée à un vecteur
*   logits to text :Le réseau neuronal traduira l'entrée en ID (mot), ce qui n'est pas la forme finale que nous voulons. La fonction logits_to_text fera le pont entre les ids du réseau neuronal et les réponses.



In [ ]:
def toPredectible(Qes, tok, shape):
    tt = []
    for q in Qes.split(' '):
        if q in tok.word_index:
            tt.append(tok.word_index[q])
        else:
            tt.append(0)
    return pad([tt], shape)
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

# Bleu score

In [ ]:
def logits_to_textx(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''
    return ' '.join([index_to_words[int(prediction)] for prediction in logits]).replace('<start>', '').replace('<end>','')

#-------------------------------------------------------------------------
preds_text = [logits_to_text(models2s.predict([[q]])[0], A_tokenizer).replace('<start>','').replace('<end>','').replace('<PAD>','') for q in X_test]
y_test_xt = [logits_to_textx(y, A_tokenizer) for y in Y_test]

#-------------------------------------------------------------------------

from nltk.translate.bleu_score import sentence_bleu
bleu_score = []
bleu_score1 = []
bleu_score2 = []
bleu_score3 = []
for i in range(len(preds_text)):
    scoret = sentence_bleu([str(y_test_xt[i]).split()], str(preds_text[i]).split(), weights=(1, 0, 0, 0))
    score1 = sentence_bleu([str(y_test_xt[i]).split()], str(preds_text[i]).split(), weights=(0.5, 0.5, 0, 0))
    score2 = sentence_bleu([str(y_test_xt[i]).split()], str(preds_text[i]).split(), weights=(0.33, 0.33, 0.33, 0))
    score3 = sentence_bleu([str(y_test_xt[i]).split()], str(preds_text[i]).split(), weights=(0.25, 0.25, 0.25, 0.25))
    bleu_score.append(scoret)
    bleu_score1.append(score1)
    bleu_score2.append(score2)
    bleu_score3.append(score3)
print("The BLEU score 1-gram on our corpus is about {}".format(sum(bleu_score) / len(bleu_score)))
print("The BLEU score 2-gram on our corpus is about {}".format(sum(bleu_score1) / len(bleu_score1)))
print("The BLEU score 3-gram on our corpus is about {}".format(sum(bleu_score2) / len(bleu_score2)))
print("The BLEU score 4-gram on our corpus is about {}".format(sum(bleu_score3) / len(bleu_score3)))

c:\users\ucefb\appdata\local\programs\python\python38\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\ucefb\appdata\local\programs\python\python38\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\ucefb\appdata\local\programs\python\python38\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

The BLEU score 1-gram on our corpus is about 0.825744276646962
The BLEU score 2-gram on our corpus is about 0.4090389649329261
The BLEU score 3-gram on our corpus is about 0.21428938053105573
The BLEU score 4-gram on our corpus is about 0.09952761357694892
